# NLTK 영화 리뷰 데이터 준비

주의점 : `tenorflow` 2.6.0 부터 `keras`가 분리 되었다

In [ ]:
import nltk
nltk.download('movie_reviews')

In [11]:
from nltk.corpus import movie_reviews

fileids = movie_reviews.fileids()   # movie review data에서 fileid를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids]     # file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]    # file id를 이용해 label로 사용할 category 즉 positive와 negative 정보를 순서대로 가져옴

print('Reviews count:', len(reviews))
print('Length of the first review:', len(reviews[0]))
print('Labels:', set(categories))

Reviews count: 2000
Length of the first review: 4043
Labels: {'neg', 'pos'}


**토크나이징 및 데이터 구축**

In [12]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer

np.random.seed(7)
tf.random.set_seed(7)

max_words = 10000   # 사용할 단어의 수

tokenizer = Tokenizer(num_words=max_words, oov_token='UNK')     # 빈도가 높은 10000개의 단어를 선택하도록 객체 생성

tokenizer.fit_on_texts(reviews)     # 단어 인덱스 구축

X = tokenizer.texts_to_sequences(reviews)   # 만들어진 단어 인덱스를 이용해 문서들을 변환

print('Lengths of first 10 documents:', [len(doc) for doc in X[:10]])

Lengths of first 10 documents: [710, 240, 486, 479, 733, 671, 550, 584, 702, 776]


In [13]:
print("Index of 'the':", tokenizer.word_index["the"])
print("Index of 'review':", tokenizer.word_index["review"])
print("Index of out-of-vocabulary words:", tokenizer.word_index["UNK"])

Index of 'the': 2
Index of 'review': 466
Index of out-of-vocabulary words: 1


In [17]:
from keras.utils import pad_sequences   # keras.preprocessing.sequence -> keras.utils로 옮겨짐

maxlen = 500    # 문서의 단어 수를 제한

# 문서의 단어 수가 500보다 작으면 0을 채우고, 크면 잘라냄
# 잘라낼 때, 앞을 자를 지, 뒤를 자를 지 선택할 수 있음. 앞을 자르고 뒷부분을 쓰고자 하면 'pre' 선택
X = pad_sequences(X, maxlen=maxlen, truncating='pre') 

In [18]:
# label을 0, 1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categories])
print(set(y))

{0, 1}


**훈련, 테스트 데이터 분리**

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print('Train set count:', len(X_train))
print('Test set count:', len(X_test))
print('Test samples:', y_test[:20])

Train set count: 1600
Test set count: 400
Test samples: [0 1 1 0 1 1 0 0 0 0 1 1 0 0 1 0 0 1 1 1]
